In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
import torch.nn.functional as F
import math

from torch.optim import lr_scheduler

In [3]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:

# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [5]:
def squash(x):
    lengths2 = x.pow(2).sum(dim=2)
    lengths = lengths2.sqrt()
    x = x * (lengths2 / (1 + lengths2) / lengths).view(x.size(0), x.size(1), 1)
    return x

In [6]:
class Encode_layer(nn.Module):
    def __init__(self, input_dim, output_dim, output_caps):
        super(Encode_layer, self).__init__()
        
        self.layer = nn.Linear(input_dim, output_dim*output_caps)
        self.output_dim = output_dim
        self.output_caps = output_caps
        
    def forward(self, input) : 
        output = self.layer(input)
        output = F.relu(output)
        output = output.view(output.shape[0], self.output_caps, self.output_dim)
        output = squash(output)
        
        return output

In [7]:
class Neuron_layer(nn.Module):
    def __init__(self, input_neurons, input_dim, output_dim, output_neurons):
        super(Neuron_layer, self).__init__()
        self.input_neurons = input_neurons
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.output_neurons = output_neurons
        self.weights = nn.Parameter(torch.Tensor(input_neurons, input_dim, output_neurons * output_dim))
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.input_neurons)
        self.weights.data.uniform_(-stdv, stdv)
        
    def forward(self, input):
        
        input = input.unsqueeze(2)
        predict = input.matmul(self.weights)
        predict = predict.view(predict.size(0), self.input_neurons, self.output_neurons, self.output_dim)
        
        return predict
        

In [8]:
map_shape = [4, 3, 10]
leng = 3
bs = []
batch_size=100

for (i, j) in zip(map_shape[:-1], map_shape[1:]):
    bs.append(torch.zeros((batch_size, i,j)))
        
def backprop(reward) :
        
    bs[1]=reward[1].data
    
    for k in range(batch_size):
        for state in range(bs[0].shape[1]) :
            max_value, _ = torch.max(bs[1][k], dim=1)
            bs[0][k][state] =reward[0].data[k][state] + max_value
                

In [9]:
def train(epoch):
    model.train()
    correct=0.
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data.view(data.shape[0], -1)), Variable(target, requires_grad=False)
        optimizer.zero_grad()
        probs, reward = model(data, bs)
        print (probs[0])
        
        backprop(reward)
        
        loss = loss_fn(probs, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data[0]))
            
        pred = probs.data.max(1, keepdim=True)[1]  # get the index of the max probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    print('Accuracy : {:.2f}%'.format(100. * correct / len(train_loader.dataset)))

In [10]:
class Learning(nn.Module) :
    def __init__(self) :
        super(Learning, self).__init__()
        
        self.network1 = Encode_layer(784, 128, 4)
        self.network2 = Neuron_layer(4, 128, 32, 3)
        self.network3 = Neuron_layer(3, 32, 16, 10)
        
    def forward(self, input, bs):
        
        self.bs=bs
        output1 = self.network1(input)
        output2 = self.network2(output1)
        
        c2=F.softmax(nn.Parameter(self.bs[0].view(-1, self.bs[0].shape[2])), dim=1).view(-1, self.bs[0].shape[1], self.bs[0].shape[2], 1)
        
        s2 = (c2*output2).sum(dim=1)
        v2 = squash(s2)
        
        v2_hat = v2.unsqueeze(1)
        reward0 = torch.squeeze((output2 * v2_hat).sum(-1))
        
        output3 = self.network3(v2)
        
        c3=F.softmax(nn.Parameter(self.bs[1].view(-1, self.bs[1].shape[2])), dim=1).view(-1, self.bs[1].shape[1], self.bs[1].shape[2], 1)
        s3 = (c3*output3).sum(dim=1)
        v3 = squash(s3)
        
        v3_hat = v3.unsqueeze(1)
        reward1 = torch.squeeze((output3 * v3_hat).sum(-1))
        reward = [reward0, reward1]
        
        probs = v3.pow(2).sum(dim=2).sqrt()
        
        return probs, reward

In [11]:
model=Learning()

In [12]:
class MarginLoss(nn.Module):
    def __init__(self, m_pos, m_neg, lambda_):
        super(MarginLoss, self).__init__()
        self.m_pos = m_pos
        self.m_neg = m_neg
        self.lambda_ = lambda_

    def forward(self, lengths, targets, size_average=True):
        t = torch.zeros(lengths.size()).long()
        if targets.is_cuda:
            t = t.cuda()
        t = t.scatter_(1, targets.data.view(-1, 1), 1)
        targets = Variable(t)
        losses = targets.float() * F.relu(self.m_pos - lengths).pow(2) + \
                 self.lambda_ * (1. - targets.float()) * F.relu(lengths - self.m_neg).pow(2)
        return losses.mean() if size_average else losses.sum()


In [13]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=15, min_lr=1e-6)

loss_fn = MarginLoss(0.9, 0.1, 0.5)

In [14]:
import numpy as np

In [15]:
for epoch in range(10) :
    print(epoch)
    train(epoch)

0
Variable containing:
1.00000e-03 *
  8.6983
  4.2593
  5.1679
  4.5620
  4.6936
  6.7221
  5.6714
  2.9449
  2.7180
  5.4882
[torch.FloatTensor of size 10]

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.079832
Variable containing:
1.00000e-03 *
  7.0537
  3.4790
  3.6945
  6.2167
  3.4981
  6.5160
  4.9846
  3.8377
  7.1925
  6.1685
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  0.9473
  0.8648
  1.1702
  1.1113
  0.5198
  1.7610
  0.5103
  0.7184
  0.6585
  0.9823
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  0.9455
  1.0468
  1.1966
  1.0177
  0.3718
  1.6560
  0.8144
  1.1593
  0.9990
  1.0638
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  1.3394
  1.0825
  0.8026
  1.4853
  1.2090
  2.4114
  1.4033
  1.4035
  1.0021
  1.1862
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  1.1443
  0.6582
  1.4140
  1.0256
  0.9889
  2.1618
  0.8942
  1.1528
  0.9790
  2.1910
[torch.FloatTensor of size 10]

Variable co

Variable containing:
 0.1494
 0.0342
 0.0441
 0.0816
 0.0555
 0.0818
 0.0723
 0.0872
 0.0451
 0.0876
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  8.1612
  3.3617
  5.6650
  8.0139
  5.8496
  7.5650
  9.9048
  7.0936
  4.2307
  9.0386
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  5.3423
  8.3713
  5.0673
  7.0485
  3.8784
  7.6291
  5.5534
  5.0450
  5.0411
  7.0228
[torch.FloatTensor of size 10]

Variable containing:
 0.0781
 0.0651
 0.0626
 0.1195
 0.0536
 0.0974
 0.0656
 0.0837
 0.0622
 0.0929
[torch.FloatTensor of size 10]

Variable containing:
 0.0800
 0.0817
 0.0561
 0.1107
 0.0500
 0.1000
 0.0574
 0.0799
 0.0596
 0.0918
[torch.FloatTensor of size 10]

Variable containing:
 0.0662
 0.0484
 0.0517
 0.1140
 0.0508
 0.0805
 0.0548
 0.0770
 0.0545
 0.0942
[torch.FloatTensor of size 10]

Variable containing:
 0.0564
 0.0590
 0.0522
 0.0901
 0.0510
 0.0772
 0.0595
 0.0757
 0.0481
 0.1010
[torch.FloatTensor of size 10]

Train Epoch: 0 [6000/

Variable containing:
 0.0861
 0.0800
 0.0772
 0.1211
 0.0592
 0.0958
 0.1474
 0.0890
 0.0704
 0.0788
[torch.FloatTensor of size 10]

Variable containing:
 0.0554
 0.1732
 0.0672
 0.0929
 0.0617
 0.0866
 0.0767
 0.1156
 0.0700
 0.1088
[torch.FloatTensor of size 10]

Variable containing:
 0.0761
 0.1343
 0.0606
 0.0820
 0.0863
 0.1075
 0.0640
 0.1505
 0.0702
 0.1301
[torch.FloatTensor of size 10]

Variable containing:
 0.0950
 0.0897
 0.0652
 0.1037
 0.0965
 0.1176
 0.0820
 0.1541
 0.0953
 0.1442
[torch.FloatTensor of size 10]

Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.056532
Variable containing:
 0.0459
 0.1112
 0.0458
 0.0787
 0.1127
 0.1015
 0.0804
 0.1279
 0.0745
 0.1338
[torch.FloatTensor of size 10]

Variable containing:
 0.0527
 0.0860
 0.0486
 0.0622
 0.1324
 0.0922
 0.0626
 0.1994
 0.0552
 0.1606
[torch.FloatTensor of size 10]

Variable containing:
 0.0744
 0.0733
 0.0815
 0.0939
 0.0793
 0.0971
 0.1574
 0.1128
 0.0575
 0.1044
[torch.FloatTensor of size 10]

Variable containing

Variable containing:
 0.0840
 0.0395
 0.0462
 0.0529
 0.1439
 0.0641
 0.0670
 0.1902
 0.0548
 0.1704
[torch.FloatTensor of size 10]

Variable containing:
 0.0896
 0.0926
 0.0458
 0.0947
 0.1020
 0.1585
 0.0789
 0.1141
 0.1044
 0.1227
[torch.FloatTensor of size 10]

Variable containing:
 0.0702
 0.1121
 0.0622
 0.1539
 0.0782
 0.1540
 0.0496
 0.1367
 0.1280
 0.1275
[torch.FloatTensor of size 10]

Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.050858
Variable containing:
 0.0494
 0.0576
 0.0764
 0.1053
 0.1382
 0.1009
 0.1045
 0.1114
 0.1200
 0.1669
[torch.FloatTensor of size 10]

Variable containing:
 0.0511
 0.2304
 0.0716
 0.0772
 0.0586
 0.0913
 0.0663
 0.0622
 0.0882
 0.0937
[torch.FloatTensor of size 10]

Variable containing:
 0.0916
 0.1407
 0.0774
 0.2287
 0.0510
 0.1174
 0.0831
 0.1056
 0.0935
 0.0714
[torch.FloatTensor of size 10]

Variable containing:
 0.1267
 0.0734
 0.0872
 0.3043
 0.0594
 0.1224
 0.1076
 0.0807
 0.1307
 0.0825
[torch.FloatTensor of size 10]

Variable containing

Variable containing:
 0.0655
 0.1006
 0.0850
 0.0659
 0.1449
 0.0968
 0.0518
 0.2379
 0.0690
 0.1462
[torch.FloatTensor of size 10]

Variable containing:
 0.0537
 0.0629
 0.0462
 0.0260
 0.2925
 0.0829
 0.0602
 0.2541
 0.0756
 0.2270
[torch.FloatTensor of size 10]

Variable containing:
 0.2263
 0.0408
 0.0540
 0.0823
 0.0671
 0.0950
 0.0557
 0.0847
 0.0739
 0.0570
[torch.FloatTensor of size 10]

Variable containing:
 0.4198
 0.0312
 0.0708
 0.0633
 0.0284
 0.0731
 0.0970
 0.0528
 0.0672
 0.0239
[torch.FloatTensor of size 10]

Variable containing:
 0.3124
 0.0886
 0.1015
 0.2215
 0.0365
 0.1045
 0.0907
 0.0464
 0.0926
 0.0460
[torch.FloatTensor of size 10]

Variable containing:
 0.1580
 0.0396
 0.1032
 0.1006
 0.0551
 0.0737
 0.2391
 0.0494
 0.0681
 0.0435
[torch.FloatTensor of size 10]

Variable containing:
 0.2567
 0.0420
 0.1128
 0.0463
 0.0438
 0.0682
 0.2329
 0.1084
 0.0515
 0.0697
[torch.FloatTensor of size 10]

Variable containing:
 0.1674
 0.0577
 0.2015
 0.0779
 0.0446
 0.0713


...

Variable containing:
 0.7340
 0.0018
 0.0042
 0.0035
 0.0013
 0.0071
 0.0029
 0.0013
 0.0046
 0.0025
[torch.FloatTensor of size 10]

Variable containing:
 2.3607e-01
 5.1057e-06
 2.7518e-06
 5.0787e-06
 8.2099e-05
 5.3301e-06
 3.1062e-03
 5.9238e-06
 6.8456e-06
 5.4048e-06
[torch.FloatTensor of size 10]

Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.047981
Variable containing:
 0.0044
 0.0026
 0.0087
 0.0022
 0.0024
 0.0075
 0.7248
 0.0028
 0.0046
 0.0024
[torch.FloatTensor of size 10]

Variable containing:
 2.6659e-06
 2.7479e-06
 2.5533e-05
 2.9138e-03
 3.9008e-06
 2.3649e-04
 1.5525e-01
 4.8387e-06
 5.4702e-06
 3.7889e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0030
 0.0027
 0.0021
 0.0024
 0.0043
 0.0034
 0.7333
 0.0039
 0.0049
 0.0039
[torch.FloatTensor of size 10]

Variable containing:
 2.3289e-06
 1.7583e-05
 5.3299e-06
 6.3032e-06
 4.9344e-06
 1.0177e-05
 3.2709e-01
 4.3231e-06
 2.2239e-03
 5.9251e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.7606
 0.

Variable containing:
 0.0000
 0.0000
 0.0400
 0.0001
 0.0000
 0.0001
 0.0003
 0.0000
 0.0003
 0.2944
[torch.FloatTensor of size 10]

Variable containing:
 0.0035
 0.0022
 0.0118
 0.0033
 0.0038
 0.0048
 0.6754
 0.0042
 0.0693
 0.0206
[torch.FloatTensor of size 10]

Variable containing:
 0.0000
 0.0000
 0.0000
 0.0001
 0.0001
 0.0067
 0.3802
 0.0000
 0.0003
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 0.0033
 0.0032
 0.0039
 0.0041
 0.0028
 0.0076
 0.0053
 0.0021
 0.6750
 0.0043
[torch.FloatTensor of size 10]

Variable containing:
 0.0000
 0.0160
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.4980
 0.0001
[torch.FloatTensor of size 10]

Train Epoch: 9 [38000/60000 (63%)]	Loss: 0.051603
Variable containing:
 0.0022
 0.0021
 0.0035
 0.0051
 0.0027
 0.0066
 0.0021
 0.0024
 0.7825
 0.0051
[torch.FloatTensor of size 10]

Variable containing:
 3.2735e-03
 3.4848e-06
 4.4099e-06
 6.2954e-06
 3.8242e-06
 8.7569e-06
 4.0942e-06
 4.2478e-06
 4.9601e-01
 3.2932e-06
[torch.Floa

Variable containing:
 0.0000
 0.0000
 0.3103
 0.0001
 0.0000
 0.0021
 0.0000
 0.0001
 0.0001
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 0.0046
 0.0032
 0.0037
 0.0060
 0.0084
 0.4486
 0.0070
 0.0288
 0.0644
 0.0175
[torch.FloatTensor of size 10]

Variable containing:
 0.0004
 0.0005
 0.0003
 0.0014
 0.0002
 0.3669
 0.0003
 0.0751
 0.0004
 0.0017
[torch.FloatTensor of size 10]

Variable containing:
 0.0024
 0.0033
 0.0036
 0.0108
 0.0027
 0.0306
 0.0021
 0.6620
 0.0028
 0.0031
[torch.FloatTensor of size 10]

Variable containing:
 1.1927e-05
 8.1137e-06
 2.1751e-05
 6.7826e-06
 1.0147e-05
 6.6780e-05
 1.6207e-03
 3.1902e-01
 5.9159e-05
 1.0008e-05
[torch.FloatTensor of size 10]

Variable containing:
 0.0020
 0.0036
 0.0031
 0.4706
 0.0028
 0.0803
 0.0020
 0.0102
 0.0055
 0.0063
[torch.FloatTensor of size 10]

Train Epoch: 9 [44000/60000 (73%)]	Loss: 0.044545
Variable containing:
 0.0002
 0.0002
 0.0001
 0.1960
 0.0199
 0.0002
 0.0002
 0.0002
 0.0006
 0.0021
[torch.Floa

Variable containing:
 9.2550e-06
 8.9878e-06
 1.0954e-02
 1.4494e-05
 1.6992e-01
 2.0138e-05
 8.0231e-06
 2.0700e-05
 3.8115e-05
 9.5741e-05
[torch.FloatTensor of size 10]

Variable containing:
 0.0024
 0.7135
 0.0027
 0.0167
 0.0025
 0.0019
 0.0014
 0.0014
 0.0063
 0.0026
[torch.FloatTensor of size 10]

Variable containing:
 3.0541e-06
 1.4916e-01
 3.2004e-03
 7.5108e-06
 8.3855e-06
 8.9315e-06
 4.7454e-06
 4.2756e-06
 6.4064e-06
 8.7648e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0024
 0.0022
 0.0052
 0.0037
 0.0035
 0.0026
 0.0029
 0.7110
 0.0029
 0.0204
[torch.FloatTensor of size 10]

Variable containing:
 3.7454e-03
 3.6106e-06
 6.3822e-06
 5.6418e-06
 4.1738e-06
 4.8872e-05
 5.6333e-06
 1.5414e-01
 7.5455e-06
 3.9526e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0026
 0.0022
 0.0033
 0.0027
 0.0371
 0.0031
 0.6939
 0.0031
 0.0066
 0.0076
[torch.FloatTensor of size 10]

Variable containing:
 1.0313e-05
 5.8562e-06
 6.4912e-03
 1.0672e-05
 1.5895e-05
 

Variable containing:
 3.5956e-06
 1.6295e-06
 4.1167e-06
 5.6724e-01
 7.0791e-05
 2.4145e-05
 6.6818e-06
 1.3782e-04
 6.1198e-05
 3.5550e-03
[torch.FloatTensor of size 10]

Variable containing:
 0.0014
 0.0042
 0.0040
 0.8589
 0.0031
 0.0220
 0.0017
 0.0057
 0.0200
 0.0036
[torch.FloatTensor of size 10]

Train Epoch: 9 [55000/60000 (92%)]	Loss: 0.043741
Variable containing:
 1.3862e-05
 5.9182e-06
 8.7799e-06
 3.1720e-01
 3.3419e-06
 1.0476e-05
 4.2847e-06
 4.3351e-03
 3.1935e-06
 1.7541e-04
[torch.FloatTensor of size 10]

Variable containing:
 0.0014
 0.0015
 0.0059
 0.0063
 0.0563
 0.0077
 0.8015
 0.0010
 0.0021
 0.0024
[torch.FloatTensor of size 10]

Variable containing:
 2.5678e-03
 2.8063e-07
 1.4860e-06
 7.3038e-07
 5.9682e-07
 2.3701e-06
 1.3528e-01
 4.3351e-07
 1.9082e-06
 2.8920e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0026
 0.0027
 0.0086
 0.0048
 0.4498
 0.0059
 0.0019
 0.0033
 0.0240
 0.6304
[torch.FloatTensor of size 10]

Variable containing:
 0.0077
 0.

In [16]:
#Test
total=0
correct=0.
#batch size 다를 땐 테스트가 안 됌?
for images, labels in test_loader:
    images = Variable(images.view(images.shape[0], -1))
    outputs, rewards = model(images, bs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))


Test Accuracy of the model on the 10000 test images: 52 %


In [17]:
adv_test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1, 
                                          shuffle=False)

In [24]:
dummy=1
s=0.
t=0.

test_b0=bs[0][99].unsqueeze(0)
test_b1=bs[1][99].unsqueeze(0)

test_bs=[test_b0, test_b1]


for (x, y) in adv_test_loader :
    x, y = Variable(x, requires_grad=True), Variable(y, requires_grad=False)
    probs, _ = model(x.view(x.shape[0], -1), test_bs)
    y_pred = np.argmax(probs.data.numpy())
    loss = nn.CrossEntropyLoss()(probs, y)
    loss.backward()
    epsilon = 0.1 
    x_grad   = torch.sign(x.grad.data)
    x_adversarial = torch.clamp(x.data + epsilon * x_grad, 0, 1)
    adversarial_probs, _ = model(Variable(x_adversarial).view(x.shape[0], -1), test_bs)
    y_pred_adversarial = np.argmax(adversarial_probs.data.numpy())
    
    print("{0} th example ".format(dummy))
    print ("True value: "+ str(y.data.numpy()[0])+"\nPredicted value : "+str(y_pred)+ "\nAdversarial :" + str(y_pred_adversarial)+"\n" )
    
    dummy+=1
    
    if y.data.numpy()[0]!=y_pred :
        t+=1
    
    if y_pred!=y_pred_adversarial :
        s+=1
    if dummy==100:
        break
        
print("Accuracy of test_model : {0:.3f} , Adversarials : {1:.3f}".format((t/dummy)*100, (s/dummy)*100))

1 th example 
True value: 7
Predicted value : 7
Adversarial :7

2 th example 
True value: 2
Predicted value : 2
Adversarial :2

3 th example 
True value: 1
Predicted value : 1
Adversarial :2

4 th example 
True value: 0
Predicted value : 0
Adversarial :0

5 th example 
True value: 4
Predicted value : 4
Adversarial :0

6 th example 
True value: 1
Predicted value : 1
Adversarial :2

7 th example 
True value: 4
Predicted value : 4
Adversarial :8

8 th example 
True value: 9
Predicted value : 9
Adversarial :2

9 th example 
True value: 5
Predicted value : 6
Adversarial :6

10 th example 
True value: 9
Predicted value : 9
Adversarial :6

11 th example 
True value: 0
Predicted value : 0
Adversarial :0

12 th example 
True value: 6
Predicted value : 6
Adversarial :8

13 th example 
True value: 9
Predicted value : 9
Adversarial :8

14 th example 
True value: 0
Predicted value : 0
Adversarial :0

15 th example 
True value: 1
Predicted value : 1
Adversarial :3

16 th example 
True value: 5
Predi